In [12]:
import folium
from folium import Marker, GeoJson
from folium.plugins import HeatMap

import pandas as pd
import geopandas as gpd



## Making a buffer zone for the toxic release

In [8]:
releases = gpd.read_file("data/toxic_release_pennsylvania/toxic_release_pennsylvania.shp") 
releases.head()

,YEAR,CITY,COUNTY,ST,LATITUDE,LONGITUDE,CHEMICAL,UNIT_OF_ME,TOTAL_RELE,geometry
0,2016,PHILADELPHIA,PHILADELPHIA,PA,40.005901,-75.072103,FORMIC ACID,Pounds,0.160,POINT (2718560.227 256380.179)
1,2016,PHILADELPHIA,PHILADELPHIA,PA,39.920120,-75.146410,ETHYLENE GLYCOL,Pounds,13353.480,POINT (2698674.606 224522.905)
2,2016,PHILADELPHIA,PHILADELPHIA,PA,40.023880,-75.220450,CERTAIN GLYCOL ETHERS,Pounds,104.135,POINT (2676833.394 261701.856)
3,2016,PHILADELPHIA,PHILADELPHIA,PA,39.913540,-75.198890,LEAD COMPOUNDS,Pounds,1730.280,POINT (2684030.004 221697.388)
4,2016,PHILADELPHIA,PHILADELPHIA,PA,39.913540,-75.198890,BENZENE,Pounds,39863.290,POINT (2684030.004 221697.388)


In [13]:
m_1=folium.Map(location= (40,-75.2),zoom_start=10)

HeatMap(data=releases[['LATITUDE', 'LONGITUDE']], radius=15).add_to(m_1)

# Show the map
m_1


### adding a buffer zone arround the leaks

In [10]:
# Select one release location
release_loc = releases.iloc[500]
release_loc_df=pd.DataFrame({'lon':[release_loc.LONGITUDE],'lat':[release_loc.LATITUDE]})
new_gdf = gpd.GeoDataFrame(release_loc_df, geometry=gpd.points_from_xy(release_loc_df.lon, release_loc_df.lat))
new_gdf.crs = {'init' :'epsg:4326'}
new_gdf = new_gdf.to_crs(epsg=2263)



# create a 2km buffer zone around the release

two_km_buffer = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(10000)


/Users/arunraj/.local/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [16]:
# Create map with release incidents and buffer zone

m_2 = folium.Map(location=[39.9526,-75.1652], zoom_start=10)
HeatMap(data=releases[['LATITUDE', 'LONGITUDE']], radius=15).add_to(m_2)

Marker([release_loc['LATITUDE'], release_loc['LONGITUDE']],popup='release location').add_to(m_2)
    
# Plot the buffer zone on the map
GeoJson(two_km_buffer.to_crs(epsg=4326)).add_to(m_2)
m_2